<a href="https://colab.research.google.com/github/tfuenz/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
application_type_values = application_df["APPLICATION_TYPE"].value_counts()
application_type_values

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace = []
for a_type in application_type_values.index:
  if application_type_values.loc[a_type] < 200:
    application_types_to_replace.append(a_type)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
classifications = application_df["CLASSIFICATION"].value_counts()
classifications

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
gt1 = []
for cls in classifications.index:
  if classifications.loc[cls] > 1:
    gt1.append(cls)

classifications.loc[gt1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = []
for classes in classifications.index:
  if classifications.loc[classes] < 1000:
    classifications_to_replace.append(classes)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
dummy_df = pd.get_dummies(data=application_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']])
dummy_df.head()

,STATUS,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [ ]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
features = application_df['IS_SUCCESSFUL']
target = pd.concat([dummy_df, application_df['ASK_AMT']], axis=1)
# target = target.replace({0: False, 'N': False, 1: True, 'Y': True})
features = pd.DataFrame(features)
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1)

In [ ]:
features.head()

,IS_SUCCESSFUL
0,1
1,1
2,0
3,1
4,1


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=43, activation="relu", input_dim=1))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=43, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 43)                86        
                                                                 
 dense_10 (Dense)            (None, 5)                 220       
                                                                 
 dense_11 (Dense)            (None, 43)                258       
                                                                 
Total params: 564 (2.20 KB)
Trainable params: 564 (2.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
#  YOUR CODE GOES HERE
from tensorflow.keras.callbacks import ModelCheckpoint

X_train_tensor = tf.convert_to_tensor(X_train_scaled, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)

tf.keras.callbacks.ModelCheckpoint(
    filepath='model_weights_epoch_{epoch:02d}.h5',
    save_weights_only=True,
    save_freq=5,
)

fit_model = nn.fit(X_train_tensor, y_train_tensor, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: -32957.0586 - accuracy: 0.0000e+00
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: -140930.4844 - accuracy: 0.0000e+00
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: -366998.5625 - accuracy: 0.0000e+00
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: -753318.2500 - accuracy: 0.0000e+00
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: -1339819.8750 - accuracy: 0.5451
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: -2110788.0000 - accuracy: 1.0000
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: -3008667.7500 - accuracy: 1.0000
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: -4040966.0000 - accuracy: 1.0000
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: -5414678.5000 - accuracy: 1.0000
Epoch 10/100
804/

In [ ]:
# Evaluate the model using the test data
X_test_scaled = tf.convert_to_tensor(X_test_scaled, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: -3.5043e+09 - accuracy: 1.0000 - 562ms/epoch - 2ms/step
Loss: -3504322816.0, Accuracy: 1.0


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
tf.saved_model.save(nn, 'AlphabetSoupCharity.h5')